In [17]:
import pandas as pd



In [26]:
# Replace 'your_file_path.csv' with the actual path to your CSV file
file_path = 'data/Births.txt'

# Read the CSV file into a pandas DataFrame
births = pd.read_csv(file_path, delim_whitespace=True)
birth_df = pd.DataFrame(births)

# Display the DataFrame
birth_df.head()

,Year,Female,Male,Total
0,1933,1122180,1184820,2307000
1,1934,1166072,1229928,2396000
2,1935,1158000,1219000,2377000
3,1936,1148000,1207000,2355000
4,1937,1175000,1238000,2413000


In [36]:
path_array = ['data/Births.txt', 'data/bltper_1x1.txt','data/Deaths_1x1.txt','data/E0per.txt','data/Mx_1x1.txt','data/Population.txt']

dfs = {}

for i in path_array:
    file_data = pd.read_csv(i, delim_whitespace=True)
    # Get the file name without the extension
    name = i.split("/")[-1].split(".")[0]
    dfs[name] = pd.DataFrame(file_data)

births_df = dfs['Births']
bltper_df = dfs['bltper_1x1']
deaths_df = dfs['Deaths_1x1']
e0per_df = dfs['E0per']
mx_df = dfs['Mx_1x1']
population_df = dfs['Population']   

In [42]:
population_df.head()


,Year,Age,Female,Male,Total
0,1933,0,984472.26,1015361.55,1999833.81
1,1933,1,1040496.02,1064088.29,2104584.31
2,1933,2,1093043.81,1117527.14,2210570.95
3,1933,3,1107994.31,1135046.59,2243040.90
4,1933,4,1130624.43,1179513.62,2310138.05
